In [2]:
import pandas as pd
import os 

Reading somefiles which are in different location 

In [32]:
df = pd.read_csv('./SalesAnalysis/Sales_Data/Sales_April_2019.csv')
# df.head()

#lets read all files in print it 

In [36]:
files = [file for file in os.listdir('./SalesAnalysis/Sales_Data')]

for file in files:
    print(file)

Merging all files data into one data frames .just check in google (how to concate dataframes )

In [34]:
files = [file for file in os.listdir('./SalesAnalysis/Sales_Data')]

all_month_data = pd.DataFrame()

for file in files:
    df = pd.read_csv('./SalesAnalysis/Sales_Data/'+file)
    
    all_month_data = pd.concat([all_month_data,df])
    
#     print(all_month_data)

all_month_data.to_csv('all_data.csv',index=False)

read in updated data frames

In [41]:
all_data = pd.read_csv('all_data.csv')
all_data.head(50)
all_data.shape


# Augment data with additional columns 

when we are performing to take out month we see to many nan value we have to clean it 

In [45]:
# del all_data['Month']
# all_data.head(100) # we cant see 

nan_df = all_data[all_data.isna().any(axis=1)]
nan_df.head(50)
nan_df.shape

clean data #1 find nan values and delete it

In [61]:
all_data = all_data.dropna(how='all')
all_data.head()

clean data #2 find Or values and delete it 

In [70]:
all_data = all_data[all_data['Order Date'].str[0:2] !='Or']


# temp_df = all_data[all_data['Order Date'].str[0:2]=='Or']
# temp_df.tail()
# temp_df.shape # or file total of 355 entry 

#3 Convert column to the correct type

In [77]:
all_data['Quantity Ordered'] = pd.to_numeric(all_data['Quantity Ordered'])  # make int
all_data['Price Each'] = pd.to_numeric(all_data['Price Each'])        # make flaot

all_data.head()

add month column (data type is int64 convert it into string) 

In [78]:
datatype = all_data.dtypes['Order Date']
# print(datatype)


all_data['Month'] = all_data['Order Date'].str[0:2] 
#converted and indexed and sliced 
# all_data.tail() 

# '''go clean find Or and delete it before converting back to intger'''
all_data['Month'] = all_data['Month'].astype('int32')
# all_data.head()
# ValueError: invalid literal for int() with base 10: 'Or'

WHAT WAS THE BEST MONTH FOR SALES AND HOW MUCH WAS EARNED THAT MONTH?

ADD a sales column 

In [90]:
all_data['Sales'] = all_data['Quantity Ordered'] * all_data['Price Each']

# all_data.head()
# can't multiply sequence by non-int of type 'str'
# go back clean data or change dtype
# convert column to the correct type $$

Adding column for city # use .apply method

In [99]:
def get_city(address):
    return address.split(',')[1]

def get_state(address):
    return address.split(',')[2].split(' ')[1]

all_data['City'] = all_data['Purchase Address'].apply(lambda x:get_city(x)+" "+get_state(x))

# all_data['City'] = all_data['Purchase Address'].apply(lambda x:x.split(',')[1])
# all_data.head()

In [83]:
# groupby 
all_data.groupby('Month').sum()

In [89]:
import matplotlib.pyplot as plt

months = range(1,13)
result = all_data.groupby('Month').sum()

#we  just need sales data
plt.bar(months,result['Sales'])
plt.xticks(months)
plt.ylabel('Sales in USD ($)')
plt.xlabel('Month number')
plt.show()

WHAT CITY HAD THE HIGHEST NO OF SALES

In [102]:
# add city column 
# groupby 
result = all_data.groupby('City').sum()
# result

In [110]:
# city = all_data['City'].unique() # order messed here
city = [city for city, df in all_data.groupby('City')]

plt.bar(city,result['Sales'])
plt.xticks(city, rotation = 'vertical', size = 8)
plt.ylabel('Sales in USD ($)')
plt.xlabel('City name')
plt.show()
# graph shows austin have max sales but on dataframes we see san fransico have maximum sales

WHAT TIME SHOULD WE DISPLAY ADVERTISEMENT TO MAXI LIKELIHOOD OF CUSTOMERS BUYING PRODUCT

In [116]:
# first change order date  to date time format becuase we need to take out hours from time  

all_data['Order Date'] = pd.to_datetime(all_data['Order Date'])
# all_data.head()

# insert new column name it hours 

all_data['Hour'] = all_data['Order Date'].dt.hour
all_data['Minute'] = all_data['Order Date'].dt.minute
# all_data.head() 

In [125]:
all_data['Count'] = 1

hour = [hour for hour, df in all_data.groupby('Hour')]

plt.plot(hour, all_data.groupby(['Hour']).count())
plt.xticks(hour)
plt.xlabel('Hours')
plt.ylabel('No of orders')
plt.grid()
plt.show()

# all_data.groupby(['Hour']).count()

WHAT ORDER AREA MOST SOLD TOGETHER

In [152]:
# df = all_data[all_data['Order ID'].duplicated(keep=False)]

# x = all_data['Order ID'].duplicated(keep=False) # it ll provided you in boolean only 

# all_data[x] # it ll provide whole data frame with provided condition

# all_data['Order ID'].duplicated() # keep = "last" is a default format 
# x=all_data['Order ID'].duplicated(keep = "first") # it ll give you true first duplicate file and false rest over

In [202]:
df = all_data[all_data['Order ID'].duplicated(keep=False)] 

df['Grouped'] = df.groupby('Order ID')['Product'].transform(lambda x:','.join(x))

df = df[['Order ID','Grouped']].drop_duplicates() # grouping out duplicated 

In [169]:
#lets count the pair which appear most frequently we have to use library itertools
# https://stackoverflow.com/questions/52195887/counting-unique-pairs-of-numbers-into-a-python-dictionary

from itertools import combinations
from collections import Counter

In [173]:
count = Counter()
for row in df['Grouped']:
    row_list = row.split(',')
    count.update(Counter(combinations(row_list,2)))
    
# print(count)

In [180]:
count.most_common(10)

for key ,value in count.most_common(10):
    print(key,value)
    print(f'{key} = {value}')

WHAT PRODUCT SOLD THE MOST AND WHY DO YOU THINK IT SOLD THE MOST 

In [201]:
# all_data.head()

product_group = all_data.groupby('Product').sum()

In [200]:
# all_data.head()

product_group = all_data.groupby('Product')

quantity_ordered = product_group.sum()['Quantity Ordered']
# quantity_ordered # it think it shows total unit sold

products= [ product for product, df in product_group]

plt.ylabel('Quantity Ordered')
plt.xlabel('products name')

plt.bar(products,quantity_ordered)

plt.xticks(products,rotation = 'vertical',size=8)

plt.show()

In [199]:
prices = all_data.groupby('Product').mean()['Price Each']

fig, ax1 = plt.subplots()

ax2 = ax1.twinx()
ax1.bar(products,quantity_ordered, color='g')
ax2.plot(products,prices, 'b-')

ax1.set_xlabel('product name')
ax1.set_ylabel('quantity ordered', color='g')
ax2.set_ylabel('price ($)', color='b')